<a href="https://colab.research.google.com/github/subramanya4shenoy/MachineLearningNbs/blob/main/Linear_Regeression_Problem-1/LR_Forecasting_Mini_Course_Sales(submission2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Problem statement** ⚔

**Dataset Description**

For this challenge, you will be predicting a full year worth of sales for various fictitious learning modules from different fictitious Kaggle-branded stores in different (real!) countries. This dataset is completely synthetic, but contains many effects you see in real-world data, e.g., weekend and holiday effect, seasonality, etc. You are given the task of predicting sales during for year 2022.

Good luck!

In [ ]:
"""
Basic setup for integrating Kaggle
Make sure the kaggle.json file is available and uploaded in session
"""
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

In [ ]:
"""
Path to the dataset from kaggle
"""
!kaggle competitions download -c playground-series-s3e19

  0% 0.00/1.18M [00:00<?, ?B/s]
100% 1.18M/1.18M [00:00<00:00, 147MB/s]


In [ ]:
"""
unzipping the files and removing the zip
"""
!unzip \*.zip && rm *.zip

Archive:  playground-series-s3e19.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 



---



---





**Task**: To predict the sales during the year 2022

**Steps:**
  1. 👁 Look at the data and learn about the data.
  2. 🎯 Decide on which model to choose.
  3. 🦖 Do Exploratory Data analysis.
  4. 🧹 Prepare the data

In [ ]:
"""
Importing packages for reading data
"""
import pandas as pd

salesdf = pd.read_csv('train.csv')
testdf = pd.read_csv('test.csv')
submissiondf = pd.read_csv('sample_submission.csv')

In [ ]:
def get_insights(df):
  print("\n\n===========dataframe==========================")
  print(df.head(3))
  print("\n================================================")

  print("\n\n===========dataframe size=====================")
  print(df.shape)
  print("\n================================================")

  print("\n\n===========dataframe column names=============")
  print(df.columns)
  print("\n================================================")

  print("\n\n===========dataframe Unique value=============")
  print(df.nunique())
  print("\n================================================")

  print("\n\n===========dataframe data types===============")
  print(df.info())
  print("\n================================================")

  print("\n\n===========dataframe descriptions=============")
  print(df.describe())
  print("\n================================================")
  print("==================================================")


In [ ]:
"""
Understanding the data
"""
# get_insights(salesdf)
# get_insights(testdf)
get_insights(submissiondf)



===========dataframe==========================
       id  num_sold
0  136950       100
1  136951       100
2  136952       100



===========dataframe size=====================
(27375, 2)



===========dataframe column names=============
Index(['id', 'num_sold'], dtype='object')



===========dataframe Unique value=============
id          27375
num_sold        1
dtype: int64



===========dataframe data types===============
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27375 entries, 0 to 27374
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   id        27375 non-null  int64
 1   num_sold  27375 non-null  int64
dtypes: int64(2)
memory usage: 427.9 KB
None



===========dataframe descriptions=============
                  id  num_sold
count   27375.000000   27375.0
mean   150637.000000     100.0
std      7902.626146       0.0
min    136950.000000     100.0
25%    143793.500000     100.0
50%    150637.000000     100.0
7

### **Insights from data (EDA)**
1. We have training set of 6 columns and  136950 rows.
  *   2 int64 columns id and num_sold.
  *   Our dependant variable is num_sold. We need to predict num_sold
  *   We have date columns which need to be converted to date time
  *   We see countr, store and product columns which are categorical data. we need to convert them into numerical representation.
  *   We would be trying one-hot encoding for product columns and store columns as they have very small uniq values.


2. For test set we have similar data but num_sold column is missing (we will be predicting that)

3. For submission set we have id and num sold column. we will be filling the data, converting to csv and submitting.

   




In [ ]:
"""
Part of EDA updating date column,
converting categorical data into numbers
one-hot-encoding

we will be doing the same for both salesdf (training + test set) and testdf
so creating methods to each operations first.
"""

"""
date-column
1. method to converting all date column to datetime
2. given " This dataset is completely synthetic, but contains many effects you see in real-world data,
    e.g., weekend and holiday effect, seasonality, etc"
3. we might need to introduce more columns based on holidays, weekends.
4. We import holiday package to get seasonal holiday of any country
"""

import holidays
import numpy as np

# Create a dictionary to store holiday classes for different countries
holiday_classes = {}

def generate_holidays(df):
  for country in df['country'].unique():
    try:
        holiday_classes[country] = getattr(holidays, country)()
    except AttributeError:
        holiday_classes[country] = None


def prepare_date_column(tempdf):
  df = tempdf.copy()

  # Converting to datetime format
  df['date'] = pd.to_datetime(df['date'])
  df['date'].astype(np.int64)

  # Check if date is weekend then True else False. [5 = saturday, 6 = sunday]
  df['Is_Weekend'] = df['date'].dt.dayofweek.isin([5, 6])

  # Converting True, False into 1, 0
  df['Is_Weekend'] = df['Is_Weekend'].astype(int)

  # Checking for country specific seasonal holidays
  df['Is_Seasonal_Holiday'] = 0

  # generating special holidays for countries
  generate_holidays(df)

  # updating the seasonal holiday column
  for index, row in df.iterrows():
    country = row['country']
    if country in holiday_classes and holiday_classes[country] is not None and row['date'] in holiday_classes[country]:
        df.at[index, 'Is_Seasonal_Holiday'] = 1

  return df

In [ ]:
"""
  Country has uniq values so we will be converting into numbers via skLearn label encoders
  Store too has the same we will be number coding them via label_encoders
  Product is also limitted and will be number coding them and doing a one-hot encoding to the column via
  get_dummies
"""
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

def prepare_columns(tempdf):
  df = tempdf.copy()
  df['country'] = label_encoder.fit_transform(df['country'])
  df['store'] = label_encoder.fit_transform(df['store'])
  dummies_df = pd.get_dummies(df['product'])
  df = pd.concat([df, dummies_df], axis=1)
  df = df.drop('product', axis = 1)
  return df

def prepare_data(df):
  df = prepare_date_column(df)
  df = prepare_columns(df)
  return df

In [ ]:
# preprosing training sales data
sales_train_df = prepare_data(salesdf)

In [ ]:
"""
Now we are able to complete our EDA and preprocessing data
Lets split the data for training and testing
"""
from sklearn.model_selection import train_test_split

indices_to_remove = np.where(sales_train_df.columns == 'num_sold')[0]
category_columns = np.delete(sales_train_df.columns, indices_to_remove)

X = sales_train_df[category_columns]
y = sales_train_df['num_sold']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_train['date'] = X_train['date'].astype(np.int64)
X_val['date'] = X_val['date'].astype(np.int64)

In [ ]:
"""
Model Creation - Basic model
"""
from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(X_train, y_train)
y_val_pred = model.predict(X_val)

In [ ]:
"""
fitting the model
"""

model.fit(X_train, y_train)
y_val_pred = model.predict(X_val)

In [ ]:
"""
test data preparing
"""
sales_test_df = prepare_data(testdf)
sales_test_df['date'] = sales_test_df['date'].astype(np.int64)

In [ ]:
"""
testing out test data
"""
y_test_pred = model.predict(sales_test_df)

In [ ]:
"""
Preparing the submission data
"""
submissiondf = sales_test_df[['id']].copy()  # Use the 'id' column as unique identifier
submissiondf['num_sold'] = y_test_pred

submissiondf.to_csv('submission.csv', index=False)